## Задача 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [3]:
movies = pd.read_csv('D:/Netology/Python/Lesson_12/ml-25m/movies.csv')
movies = movies[['movieId','title']]
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [2]:
ratings = pd.read_csv('D:/Netology/Python/Lesson_12/ml-25m/ratings.csv')

In [4]:
ratings = ratings[['movieId','rating']]

In [9]:
ratings = ratings.groupby('movieId').mean().reset_index()

In [28]:
joined = ratings.merge(movies, on = 'movieId', how = 'left')

In [30]:
cols = joined.columns.tolist()
cols[1], cols[2] = cols[2], cols[1]
joined = joined[cols]

In [32]:
def rating_class(rating):
    answer = ''
    if 0 < rating < 2:
        answer = 'low rating'
    elif 2 <= rating < 4:
        answer = 'middle rating'
    elif 4 <= rating < 4.5:
        answer = 'middle-high rating'
    elif 4.5 <= rating <= 5:
        answer = 'high rating'
    return answer

In [33]:
joined['class'] = joined['rating'].apply(rating_class)

## Задача 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {  
'Центр': ['москва', 'тула', 'ярославль'],  
'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
} 

Результат классификации запишите в отдельный столбец region.

In [53]:
keywords = pd.read_csv('keywords.csv')

In [58]:
def get_region(keyword):
    keyword = keyword.lower()
    answer = ''
    if ('москв' in keyword) | ('тул' in keyword) | ('ярославл' in keyword):
        answer = 'Центр'
    elif ('петербург' in keyword) | ('псков' in keyword) | ('мурманск' in keyword):
        answer = 'Северо-Запад'
    elif ('владивосток' in keyword) | ('сахалин' in keyword) | ('хабаровск' in keyword):
        answer = 'Дальний Восток'
    else:
        answer = 'undefined'
    return answer

In [64]:
keywords['region'] = keywords['keyword'].apply(get_region)
keywords[keywords['region'].isin(['Центр', 'Северо-Запад', 'Дальний Восток'])]

,keyword,shows,region
127,авито москва,979292,Центр
143,эхо москвы,889657,Центр
197,погода в москве,745745,Центр
370,авито ру санкт петербург,425134,Северо-Запад
414,погода в москве на 14 дней,400914,Центр
...,...,...,...
99634,северпост новости мурманской области,3606,Северо-Запад
99808,полармед мурманск запись на прием,3719,Северо-Запад
99849,малахитовая шкатулка,3650,Центр
99890,яндекс метро москва,4548,Центр


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [70]:
ratings = pd.read_csv('D:/Netology/Python/Lesson_12/ml-25m/ratings.csv')
ratings = ratings[['movieId','rating']]

,movieId,rating
0,296,5.0
1,306,3.5
2,307,5.0
3,665,5.0
4,899,3.5


In [69]:
movies = pd.read_csv('D:/Netology/Python/Lesson_12/ml-25m/movies.csv')
movies = movies[['movieId','title']]

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [94]:
joined = ratings.merge(movies, on = 'movieId', how = 'left')
joined_test = joined.head(10000)

In [81]:
years = list(range(1950,2011))

In [103]:
def get_production_year(title):
    answer = 1900
    for year in years:
        if str(year) in title:
            answer = year
            break
    return answer

In [104]:
%%time
joined['year'] = joined['title'].apply(get_production_year)

Wall time: 6min


In [114]:
joined = joined.groupby('year').mean()[['rating']]

In [115]:
joined.sort_values(by=['rating'], ascending=False).reset_index()

,year,rating
0,1957,3.999492
1,1954,3.985693
2,1972,3.956477
3,1962,3.929445
4,1952,3.895315
...,...,...
57,1990,3.435736
58,2005,3.434087
59,2000,3.420897
60,1997,3.410704
